In [33]:
clin_df = ca_all1.copy()
statin_df = statin1.copy()
lab_df = dfck.copy()
lab_name = 'ck'

if lab_name.lower() == 'ck':
    gender_map = clin_df.set_index('eid')['gender']
    lab_df['gender'] = lab_df['eid'].map(gender_map)

    # Compute flags
    lab_df['GSI'] = np.where(
        ((lab_df['gender'] == 1) & (lab_df['value'] >= 180)) |
        ((lab_df['gender'] == 0) & (lab_df['value'] >= 120)),
        1, 0
    )
    lab_df['SRM'] = np.where(
        ((lab_df['gender'] == 1) & (lab_df['value'] >= 180 * 4)) |
        ((lab_df['gender'] == 0) & (lab_df['value'] >= 120 * 4)),
        1, 0
    )
    lab_df = lab_df.drop(['gender'], axis = 1)

df1 = pd.merge(statin_df, lab_df, on='eid', how='left')
df1 = df1[(df1['start_date'] <= df1['ct_date']) & (df1['ct_date'] <= df1['end_date'])]

if lab_name.lower() == 'ck':
    df1a = df1[['eid', 'GSI', 'SRM']]
    
df1 = df1.sort_values(['eid', 'ct_date']).groupby('eid').last().reset_index()[['eid', 'value']]
df2 = pd.merge(clin_df[['eid', 'lower_bound', 'blood_date']], lab_df, on='eid', how='left')
df2 = df2[~df2['eid'].isin(df1['eid'])]
df2 = df2[(df2['lower_bound'] <= df2['ct_date']) & (df2['ct_date'] <= df2['blood_date'])]

if lab_name.lower() == 'ck':
    df2a = df2[['eid', 'GSI', 'SRM']]
    dfat = pd.concat([df1a, df2a])
    dfat = dfat.groupby('eid')['SRM'].max().reset_index()

df2 = df2.sort_values(['eid', 'ct_date']).groupby('eid').last().reset_index()[['eid', 'value']]

df_combined = pd.concat([df1, df2])
df_combined.columns = ['eid', lab_name]

merged = pd.merge(clin_df, df_combined, on='eid', how='left')
if lab_name.lower() == 'ck':
    merged = pd.merge(merged, dfat, on = 'eid', how = 'left')
merged

,eid,age,center,gender,bmi,PC1,PC2,edu_yrs,current_smk,log_alcohol_weekly_g,tdi,waist_hip_ratio,A10BA_bf,A02BC_bf,C10AA_bf,blood_date,lower_bound,ck,SRM
0,1,55.794943,3,0,45.782673,82.265814,-34.053560,9,0,4.734440,-2.598237,1.598545,1,0,1,2010-11-13,2007-11-13,13.229484,0.0
1,2,58.416383,3,1,54.109921,79.889365,-22.688903,3,0,5.905656,4.569431,1.522687,1,0,0,2009-12-26,2006-12-26,253.383488,0.0
2,3,49.690531,3,1,34.648029,78.911215,-0.057789,5,1,3.168368,3.652507,0.833028,1,0,0,2010-11-22,2007-11-22,208.553290,0.0
3,4,59.545991,3,1,37.238151,58.776280,7.346400,5,1,3.532665,-3.771664,1.021325,0,0,0,2010-11-24,2007-11-24,75.284568,0.0
4,5,53.059908,1,0,33.424896,105.098353,-32.590387,8,0,2.996336,5.306693,1.182556,1,0,1,2008-09-04,2005-09-04,113.754548,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,58.552278,1,1,37.808100,63.557898,-24.363006,6,1,3.612827,-1.254024,0.956264,1,0,1,2009-07-25,2006-07-25,NaN,NaN
96,97,52.607725,1,0,33.846139,55.116471,-30.118030,10,0,3.055583,5.827871,1.108246,1,0,1,2010-09-09,2007-09-09,249.771886,0.0
97,98,54.499790,1,1,41.190366,86.027530,-11.977128,9,1,4.088538,1.314517,1.107538,1,0,0,2009-03-06,2006-03-06,281.477362,0.0
98,99,49.991183,4,1,45.413256,49.098429,-6.945841,9,1,4.682482,4.899616,1.257251,0,0,0,2010-06-30,2007-06-30,213.430102,0.0


In [28]:
# Compute GSI/SRM only if lab is CK


,eid,ct_date,value,gender,GSI_flag,SRM_flag
0,2,2009-10-19,253.383488,1,1,0
1,4,2010-09-14,75.284568,1,0,0
2,6,2010-07-08,475.645665,0,1,0
3,14,2009-08-06,503.381492,1,1,0
4,15,2009-09-01,270.096885,1,1,0
...,...,...,...,...,...,...
80,89,2009-10-02,533.899286,1,1,0
81,90,2008-11-14,91.982373,1,0,0
82,93,2010-02-03,63.279688,0,0,0
83,97,2010-09-05,249.771886,0,1,0


In [9]:
def merge_lab_within_window(clin_df, statin_df, lab_df, lab_name):
    if lab_name.lower() == 'ck':
        gender_map = clin_df.set_index('eid')['gender']
        lab_df['gender'] = lab_df['eid'].map(gender_map)

        # Compute flags
        lab_df['GSI'] = np.where(
            ((lab_df['gender'] == 1) & (lab_df['value'] >= 180)) |
            ((lab_df['gender'] == 0) & (lab_df['value'] >= 120)),
            1, 0
        )
        lab_df['SRM'] = np.where(
            ((lab_df['gender'] == 1) & (lab_df['value'] >= 180 * 4)) |
            ((lab_df['gender'] == 0) & (lab_df['value'] >= 120 * 4)),
            1, 0
        )
        lab_df = lab_df.drop(['gender'], axis = 1)

    df1 = pd.merge(statin_df, lab_df, on='eid', how='left')
    df1 = df1[(df1['start_date'] <= df1['ct_date']) & (df1['ct_date'] <= df1['end_date'])]

    if lab_name.lower() == 'ck':
        df1a = df1[['eid', 'GSI', 'SRM']]

    df1 = df1.sort_values(['eid', 'ct_date']).groupby('eid').last().reset_index()[['eid', 'value']]
    df2 = pd.merge(clin_df[['eid', 'lower_bound', 'blood_date']], lab_df, on='eid', how='left')
    df2 = df2[~df2['eid'].isin(df1['eid'])]
    df2 = df2[(df2['lower_bound'] <= df2['ct_date']) & (df2['ct_date'] <= df2['blood_date'])]

    if lab_name.lower() == 'ck':
        df2a = df2[['eid', 'GSI', 'SRM']]
        dfat = pd.concat([df1a, df2a])
        dfat1 = dfat.groupby('eid')['GSI'].max().reset_index()
        dfat2 = dfat.groupby('eid')['SRM'].max().reset_index()
        dfat = pd.merge(dfat1, dfat2, on = 'eid')

    df2 = df2.sort_values(['eid', 'ct_date']).groupby('eid').last().reset_index()[['eid', 'value']]

    df_combined = pd.concat([df1, df2])
    df_combined.columns = ['eid', lab_name]

    merged = pd.merge(clin_df, df_combined, on='eid', how='left')
    if lab_name.lower() == 'ck':
        merged = pd.merge(merged, dfat, on = 'eid', how = 'left')

    return merged

,eid,value1,read_2,read_3,event_dt
0,2,253.383488,44HG.,NaN,2009-10-19
1,4,75.284568,44H4.,NaN,2010-09-14
2,6,475.645665,44HE.,NaN,2010-07-08
3,13,0.000000,44HG.,NaN,2009-03-20
4,14,503.381492,44H4.,NaN,2009-08-06
...,...,...,...,...,...
95,93,63.279688,NaN,44HG.,2010-02-03
96,95,0.000000,NaN,XaESB,2007-11-20
97,96,0.000000,NaN,XaESB,2009-07-22
98,97,249.771886,NaN,44HG.,2010-09-05


In [7]:
#d2 = pd.read_csv('../data/drug_data/ATC4_prescription/C10AA.csv', low_memory = False)
statin1 = pd.read_csv('../data/drug_data/ATC4_prescription/C10AA.csv', low_memory=False)
ca_all1 = ca_all.copy()

ca_all1 = merge_lab_within_window(ca_all1, statin1, dfalt, 'alt')
#ca_all1 = merge_lab_within_window(ca_all1, statin1, dfast, 'ast')
#ca_all1 = merge_lab_within_window(ca_all1, statin1, dfck, 'ck')
#ca_all1 = merge_lab_within_window(ca_all1, statin1, dfggt, 'ggt')
#ca_all1 = merge_lab_within_window(ca_all1, statin1, dfbil, 'bil')
#ca_all1 = merge_lab_within_window(ca_all1, statin1, dfalb, 'alb')
#ca_all1 = merge_lab_within_window(ca_all1, statin1, dfalp, 'alp')
ca_all1

KeyError: 'start_date'